<a href="https://colab.research.google.com/github/sanjaykmohanty/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!pip install pyspark

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 57.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=31c39009fae8b5d5c0ccc929c448e280ede5da808b317cfaaebae314cb12f946
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-24 16:06:26--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.17MB/s    in 0.2s    

2022-12-24 16:06:27 (6.17 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...|2015-08-31 00:00:00|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...| 

# Perform Analysis on Dataframe

In [5]:
# filter DataFrame for total_votes equal to or greater than 20
total_votes_df = df.filter(df.total_votes >= 20)
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2622293|R1OAZUG90XPU2U|B00JQ47DO2|     268145085|Epic Bison Bacon ...|         Grocery|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...|2015-08-31 00:00:00|
|         US|   12244195|R23VPTUJ8I8NR5|B00XWQ2F2K|     138780411|Anna and Sarah Or...| 

In [7]:
# Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = total_votes_df.filter((total_votes_df.helpful_votes / total_votes_df.total_votes) >= 0.5)
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2622293|R1OAZUG90XPU2U|B00JQ47DO2|     268145085|Epic Bison Bacon ...|         Grocery|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...|2015-08-31 00:00:00|
|         US|   12244195|R23VPTUJ8I8NR5|B00XWQ2F2K|     138780411|Anna and Sarah Or...| 

In [8]:
# Create paid vine DataFrame
paid_vine_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
paid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   34773146|R3MKO875WGIEBJ|B00N9IMNKA|      61108636|Vita Coco Organic...|         Grocery|          5|           34|         37|   Y|                N|My Top Coconut Oi...|I'm very strict w...|2015-08-05 00:00:00|
|         US|   35093059|R3OPNTK61FQ7MP|B00S0AOIFA|     713485594|    V8 Veggie Blends| 

In [9]:
# Create unpaid vine DataFrame
unpaid_vine_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
unpaid_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2622293|R1OAZUG90XPU2U|B00JQ47DO2|     268145085|Epic Bison Bacon ...|         Grocery|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...|2015-08-31 00:00:00|
|         US|   12244195|R23VPTUJ8I8NR5|B00XWQ2F2K|     138780411|Anna and Sarah Or...| 

In [10]:
# paid total number of reviews
total_paid_reviews = paid_vine_df.count()
total_paid_reviews

61

In [11]:
# paid 5-star reviews
paid_five_star_reviews = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
paid_five_star_reviews

20

In [12]:
# paid 5-star reviews percentage
paid_five_star_percentage = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percentage

32.78688524590164

In [13]:
# unpaid total number of reviews
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews

28287

In [14]:
# unpaid 5-star reviews
unpaid_five_star_reviews = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
unpaid_five_star_reviews

15689

In [15]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

55.463640541591545